In [1]:
import pandas as pd, pyodbc

In [2]:
conn_prod = pyodbc.connect('Driver={SQL Server}; Server=10.241.109.41,20010\\OASYSHDB;Database=EPSA_Reporting;UID=epsareportes; PWD=Epsa.2020!;')
conn_dev = pyodbc.connect('Driver={SQL Server}; Server=10.241.114.12,20010\\OASYSHDB;Database=ADMS_QueryEngine;UID=Epsareportes; PWD=cmXoasys2;')

In [17]:
analog_prod = pd.read_sql_query("select name, customID,powerResource,gid from ADMS_QueryEngine.dms.AnalogSignal where name like '%_CT-%'",conn_dev, coerce_float=False)
CT_Breaker = pd.read_sql_query('''select	CT.name as CurrentTransformer_Name, CT.container as BayGid, CT.gid as CurrentTransformerGid, Br.gid as BreakerGid, Br.aorGroup as BreakerAorGroup, CT.aorGroup as CurrentTransformerAorGroup,
		[Transmision] = (select TN.gid from dms.TransmissionNetwork as TN JOIN DMS.Substation as S on S.parentContainer = TN.gid where B.parentContainer = S.gid)
		 from ADMS_QueryEngine.dms.CurrentTransformer as CT
    join ADMS_QueryEngine.dms.Bay as B ON B.gid = CT.container 
    join ADMS_QueryEngine.dms.BReaker as Br on Br.container = B.gid
    union 
    select	CT.name as CurrentTransformer_Name, CT.container as BayGid, CT.gid as CurrentTransformerGid, Br.gid as BreakerGid, Br.aorGroup, CT.aorGroup,
			[Transmision] = (select TN.gid from dms.TransmissionNetwork as TN JOIN DMS.Substation as S on S.parentContainer = TN.gid where B.parentContainer = S.gid)
		 from ADMS_QueryEngine.dms.CurrentTransformer as CT
    join ADMS_QueryEngine.dms.Bay as B ON B.gid = CT.container
    join ADMS_QueryEngine.dms.PortableCell as R on R.parentContainer = B.gid 
    join ADMS_QueryEngine.dms.BReaker as Br on Br.container = R.gid

    ''',conn_dev, coerce_float=False)

In [18]:
CT_Breaker_merged = CT_Breaker[CT_Breaker.Transmision.notnull()].merge(analog_prod,left_on='BreakerGid', right_on='powerResource', how='inner')[['CurrentTransformerGid','BreakerGid','CurrentTransformerAorGroup']].drop_duplicates()

In [13]:
template = """  <Script>
    <Name>{}</Name>
    <Description />
    <Resource>1734167331516776448</Resource>
    <Properties>
      <UpdateProperty>
        <PropertyID>1729382256910271497</PropertyID>
        <Operation>{}</Operation>
        <ReferencedResource />
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1152921504606850057</PropertyID>
        <Operation>{}</Operation>
        <ReferencedResource />
      </UpdateProperty>
    </Properties>
    <Where>[Power Resource] = {}</Where>
    <Action>Update</Action>
    <TimeOfCreated>16:18:58 01/06/2022</TimeOfCreated>
  </Script>"""

In [14]:
body = """<?xml version="1.0" encoding="utf-8"?>
<ArrayOfScript xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">"""
for i,x in CT_Breaker_merged.iterrows():
    body = body + '\n'+ template.format('.',x.CurrentTransformerGid, x.CurrentTransformerAorGroup,x.BreakerGid)
body = body + '\n'+ '</ArrayOfScript>'

In [15]:
with open('.\CT2Breaker.xml', 'w+') as xmlFile:
    xmlFile.write(body)
    xmlFile.close()

In [ ]:
CT_Breaker_merged.to_csv('CT2Breaker.csv',index = False)

In [7]:
CT_Breaker[CT_Breaker.Transmision.notnull()]#_merged

,CurrentTransformer_Name,BayGid,CurrentTransformerGid,BreakerGid,BreakerAorGroup,Transmision
5,CT-T330,288793334695460424,288793450659568986,288793682587818545,292171438142522516,288793888745848839
10,CT-TA13,1318147324526327731,1318147440490436973,1318147672418686046,292171438142522569,288793888745848839
23,CT-T34,288793334695460441,288793450659568997,288793682587818559,289075213398705578,288793888745848839
30,CT-G33,1318147324526327721,1318147440490436962,1318147672418686042,292171438142522569,288793888745848839
38,CT-G31,1318147324526327718,1318147440490436964,1318147672418686039,292171438142522569,288793888745848839
...,...,...,...,...,...,...
2221,CT-T180,288793334695461326,288793450659569526,288793682587819210,292171438142522516,288793888745848839
2222,CT-4L11,751256721431066797,751256837395175984,751257069323425538,292171438142522515,288793888745848839
2225,CT-L190,288793334695459388,288793450659569376,288793682587817576,292171438142522561,288793888745848839
2229,CT-T240,288793334695461315,288793450659569522,288793682587819194,292171438142522516,288793888745848839


In [ ]:
conn_inteligrid = pyodbc.connect('Driver={SQL Server}; Server=10.241.109.41,20010\\OASYSHDB;Database=EPSA_Reporting;UID=cons_intelligrid; PWD=grid#2021;')